In [19]:
from models import *

In [20]:
board = Board(10, 10, spawn_rate=2, max_spawn_batch=3)
for i in range(100):
    board.step()

In [21]:
%%capture

all_grid = board.datacollector.get_model_vars_dataframe()
fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [22]:
all = board.datacollector.get_model_vars_dataframe()
print(all.iloc[i]['Grid'])
# Save grid to txt file
with open('grid.txt', 'w') as f:
    for i in range(len(all_grid)):
        f.write(str(all_grid.iloc[i][0]))
        f.write("\n")
        

[[4. 4. 4. 0. 0. 0. 0. 4. 4. 4.]
 [4. 4. 4. 0. 0. 0. 0. 4. 4. 4.]
 [4. 4. 4. 0. 0. 0. 0. 4. 4. 4.]
 [0. 0. 2. 2. 0. 0. 5. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 2. 3. 0. 0. 0. 0. 0.]
 [0. 2. 0. 2. 0. 0. 0. 0. 0. 0.]
 [4. 4. 4. 3. 0. 0. 0. 4. 4. 4.]
 [4. 4. 4. 0. 0. 0. 0. 4. 4. 4.]
 [4. 4. 4. 0. 0. 3. 0. 4. 4. 4.]]


In [23]:
# Success rate
print(f"Success rate: {board.successful_trips / (board.successful_trips + board.crashes)*100}%")
print(f"Crashes: {board.crashes}")
print(f"Successful trips: {board.successful_trips}")

Success rate: 94.02985074626866%
Crashes: 2
Successful trips: 31.5


In [24]:
anim

# M3. Interacciones entre agentes (Etapa Reto: 1.1)

Narhari Olalde Guajardo			| A01284077<br>
Guillermo Tafoya Milo			| A01633790<br>
Mauricio Maqueo Huerta			| A01620649<br>
Emiliano Yoltic Martinez 		| A01620000<br>

## Responder a las preguntas de la etapa 1.1 del reto:

### ¿Cómo se modela la circulación de un automóvil en un ambiente urbano?

Para el desarrollo de este proyecto el comportamiento de los vehículos estará determinado mediante inteligencia artificial e interactuando en el entorno mediante sensores ubicados en el juego de Unity en forma de semáforos.

### ¿Cómo se modela la circulación de un grupo de automóviles en un ambiente urbano?

La idea principal del proyecto es modelar el comportamiento del tráfico en un entorno controlado en el cual podamos modificar los parámetros por los cuales los semáforos se rigen al igual que poder controlar la velocidad y población de los coches. 


1. Mientras no haya un vehículo cercano, el semáforo estará en luz amarilla. 
2. Cuando un vehículo se acerque a la intersección, enviará un mensaje con el tiempo estimado de arribo.
3. El semáforo dará luz verde al semáforo más cercano y establecerá un programa de luces a partir de ese punto para el resto de los vehículos.

#### Agentes:
1. Coches
2. Semáforos

#### Diagrama de estados
1. Mientras no haya un vehículo cercano, el semáforo estará en luz amarilla. 
No vehiculo = semaforo en amarillo
2. Cuando un vehículo se acerque a la intersección, enviará un mensaje con el tiempo estimado de arribo.
Vehículo cerca = se manda a tiempo de llegada estimado a semáforo más cercano 
3. El semáforo dará luz verde al semáforo más cercano y establecerá un programa de luces a partir de ese punto para el resto de los vehículos.
